# Shifting of Price Forwad Curves (PFCs) for Energy Markets

Toy model for the spot price for electricity:
$$
S(t) = S_0 + 
    \begin{cases}
    0 & ,0 \leq h(t) < 8\\
    P_p & ,8 \leq h(t) < 11\\
    -P_{pv} & ,11\leq h(t) < 16\\
    P_p & ,16 \leq h(t) \leq 20\\
    0 & ,20 < h(t) \leq 23
    \end{cases} +
    \begin{cases}
        0 & ,1\leq d(d) \leq 5\\
        -P_{we} & ,6\leq d(t) \leq 7
    \end{cases} +
    \begin{cases}
        0 & ,m(t) \in \{4,5,6,7,8,9\}\\
        P_{W} & ,m(t) \in \{1,2,3,10,11,12\}
    \end{cases} + \varepsilon
$$
The parameters here are:
$$
\begin{align*}
S_0 &\quad\text{Spot price level}\\
P_p & \quad\text{Peak price level}\\
P_{pv} & \quad\text{Price level with regard to solar power}\\
P_{we} & \quad\text{Price level for weekends}\\
P_W & \quad\text{Price level for winter}\\
h(t) & \quad\text{Hour of the time step $t$}\\
d(t) & \quad\text{Week day of the time step $t$}\\
m(t) & \quad\text{Month of the time step $t$}\\
\varepsilon&  \sim \mathcal{N}(\mu, \sigma^2)
\end{align*}
$$

In [24]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from rivapy.instruments.energy_futures_specifications import EnergyFutureSpecifications
from rivapy.tools.scheduler import SimpleSchedule, PeakSchedule, OffPeakSchedule, GasSchedule
from rivapy.marketdata_tools.pfc_shifter import PFCShifter
from collections import defaultdict


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
a = [dt.datetime(2023,1,5), dt.datetime(2023,12,7,0,0)]
b = [dt.datetime(2023,1,5), dt.datetime(2023,12,7,0,0)]

In [15]:
a + b


[datetime.datetime(2023, 1, 5, 0, 0),
 datetime.datetime(2023, 12, 7, 0, 0),
 datetime.datetime(2023, 1, 5, 0, 0),
 datetime.datetime(2023, 12, 7, 0, 0)]

In [18]:
from rivapy.tools.enums import EnergyTimeGridStructure as ets

In [27]:
GasSchedule(dt.datetime(2023,1,5), dt.datetime(2023,12,7,0,0))

In [ ]:
np.random.seed(42)
def spot_price_model(timestamp: dt.datetime, spot_price_level: float, peak_price_level:float, solar_price_level: float,
                     weekend_price_level:float, winter_price_level: float, epsilon_mean: float = 0, epsilon_var:float = 1):
                    spot_price = spot_price_level
                    if (timestamp.hour >= 8 and timestamp.hour < 11) or (timestamp.hour >= 16 and timestamp.hour <= 20):
                            spot_price += peak_price_level
                    elif timestamp.hour >= 11 and timestamp.hour < 16:
                            spot_price -= solar_price_level
                    
                    if timestamp.weekday() >= 5:
                            spot_price -= weekend_price_level

                    if timestamp.month in {1,2,3,10,11,12}:
                            spot_price += winter_price_level 

                    spot_price += np.random.normal(loc=epsilon_mean, scale=np.sqrt(epsilon_var))
                    return spot_price

In [ ]:
parameter_dict = {
    'spot_price_level': 100,
    'peak_price_level': 10,
    'solar_price_level': 8,
    'weekend_price_level': 10,
    'winter_price_level': 20,
    'epsilon_mean': 0,
    'epsilon_var': 5
}


In [ ]:
date_range = pd.date_range(start='1/1/2023', end='1/1/2025', freq='H', inclusive='left')

In [ ]:
spot_prices = list(map(lambda x: spot_price_model(x, **parameter_dict), date_range))

In [ ]:
plt.plot(date_range, spot_prices)

In [ ]:
# compute the shape 
df = pd.DataFrame(data=spot_prices, index=date_range, columns=['Spot'])
df

In [ ]:
base_y = df.resample('Y').mean()
base_y.index = base_y.index.strftime('%Y')

In [ ]:
df_spot = df.copy()
df_spot.index = df_spot.index.strftime('%Y')
df_spot

In [ ]:
shape = df_spot.divide(base_y, axis='index')
shape_df = pd.DataFrame(data=shape['Spot'].tolist(), index=date_range, columns=['shape'])
shape_df

In [ ]:
shape_df.plot()

In [ ]:
contracts_schedules = {
    'off_peak': {
        'Cal23_OffPeak': SimpleSchedule(dt.datetime(2023,1,1), dt.datetime(2024,1,1), freq='1H', hours=[0,1,2,3,4,5,6,7,20,21,22,23], ignore_hours_for_weekdays=[5,6]),
        # 'Cal24_OffPeak': SimpleSchedule(dt.datetime(2024,1,1), dt.datetime(2025,1,1), freq='1H', hours=[0,1,2,3,4,5,6,7,20,21,22,23], ignore_hours_for_weekdays=[5,6]),
        # 'Q1/23_OffPeak': SimpleSchedule(dt.datetime(2023,1,1), dt.datetime(2023,4,1), freq='1H', hours=[0,1,2,3,4,5,6,7,20,21,22,23], ignore_hours_for_weekdays=[5,6]),
        'Q2/23_OffPeak': SimpleSchedule(dt.datetime(2023,4,1), dt.datetime(2023,7,1), freq='1H', hours=[0,1,2,3,4,5,6,7,20,21,22,23], ignore_hours_for_weekdays=[5,6]),
        # 'Q3/23_OffPeak': SimpleSchedule(dt.datetime(2023,7,1), dt.datetime(2023,10,1), freq='1H', hours=[0,1,2,3,4,5,6,7,20,21,22,23], ignore_hours_for_weekdays=[5,6]),
        # 'Q4/23_OffPeak': SimpleSchedule(dt.datetime(2023,10,1), dt.datetime(2024,1,1), freq='1H', hours=[0,1,2,3,4,5,6,7,20,21,22,23], ignore_hours_for_weekdays=[5,6]),
        # 'M1/23_OffPeak': SimpleSchedule(dt.datetime(2023,1,1), dt.datetime(2023,2,1), freq='1H', hours=[0,1,2,3,4,5,6,7,20,21,22,23], ignore_hours_for_weekdays=[5,6]),
        # 'M2/23_OffPeak': SimpleSchedule(dt.datetime(2023,2,1), dt.datetime(2023,3,1), freq='1H', hours=[0,1,2,3,4,5,6,7,20,21,22,23], ignore_hours_for_weekdays=[5,6]),
        # 'M3/23_OffPeak': SimpleSchedule(dt.datetime(2023,3,1), dt.datetime(2023,4,1), freq='1H', hours=[0,1,2,3,4,5,6,7,20,21,22,23], ignore_hours_for_weekdays=[5,6]),
        # 'M5/23_OffPeak': SimpleSchedule(dt.datetime(2023,5,1), dt.datetime(2023,6,1), freq='1H', hours=[0,1,2,3,4,5,6,7,20,21,22,23], ignore_hours_for_weekdays=[5,6]),
    },
    'peak': {
        'Cal23_Peak': SimpleSchedule(dt.datetime(2023,1,1), dt.datetime(2024,1,1), freq='1H', hours=[8,9,10,11,12,13,14,15,16,17,18,19], weekdays=[0,1,2,3,4]),
        'Cal24_Peak': SimpleSchedule(dt.datetime(2024,1,1), dt.datetime(2025,1,1), freq='1H', hours=[8,9,10,11,12,13,14,15,16,17,18,19], weekdays=[0,1,2,3,4]),
        'Q1/23_Peak': SimpleSchedule(dt.datetime(2023,1,1), dt.datetime(2023,4,1), freq='1H', hours=[8,9,10,11,12,13,14,15,16,17,18,19], weekdays=[0,1,2,3,4]),
        'Q2/23_Peak': SimpleSchedule(dt.datetime(2023,4,1), dt.datetime(2023,7,1), freq='1H', hours=[8,9,10,11,12,13,14,15,16,17,18,19], weekdays=[0,1,2,3,4]),
        'Q3/23_Peak': SimpleSchedule(dt.datetime(2023,7,1), dt.datetime(2023,10,1), freq='1H', hours=[8,9,10,11,12,13,14,15,16,17,18,19], weekdays=[0,1,2,3,4])
    }
}

In [ ]:
contracts = defaultdict(dict)
for contract_type, contracts_dict in contracts_schedules.items():
    for contract_name, schedule in contracts_dict.items():
        tg = schedule.get_schedule()
        price = df.loc[tg,:].mean()[0]
        contracts[contract_type][contract_name] = EnergyFutureSpecifications(schedule=schedule, price=price, name=contract_name)

In [ ]:
dict(contracts)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(date_range, spot_prices)
ax = plt.gca()
for contract_type, contract_dict in contracts.items():
    for name, contract in contract_dict.items():
        y_value = contract.price  # The y-coordinate for the horizontal line
        x_range = contract.schedule.get_schedule()  # List of datetime values
        
        # Extract the minimum and maximum dates from the x_range
        xmin = min(x_range)
        xmax = max(x_range)
        
        # Plot a horizontal line for the contract
        color = next(ax._get_lines.prop_cycler)['color']
        plt.hlines(y=y_value, xmin=xmin, xmax=xmax, linestyle='--', label=name, colors=color)

plt.legend()
plt.show()

In [ ]:
shape_df

In [ ]:
shape_df_off_peak = shape_df.loc[SimpleSchedule(dt.datetime(2023,1,1), dt.datetime(2024,1,1), freq='1H', hours=[0,1,2,3,4,5,6,7,20,21,22,23], ignore_hours_for_weekdays=[5,6]).get_schedule()]
shape_df_peak = shape_df.loc[SimpleSchedule(dt.datetime(2023,1,1), dt.datetime(2025,1,1),freq='1H', hours=[8,9,10,11,12,13,14,15,16,17,18,19], weekdays=[0,1,2,3,4]).get_schedule()]

In [ ]:
shape_df_off_peak

In [ ]:
pfc_shifter = PFCShifter(shape=shape_df_off_peak, contracts=contracts['off_peak'])

In [ ]:
pfc_shifter

In [ ]:
pfc_shifter.shape

In [ ]:
pfc_shifter._get_contract_start_end_dates()

In [ ]:
transition_matrix=pfc_shifter.generate_transition_matrix()
transition_matrix

In [ ]:
transition_matrix = pfc_shifter.detect_redundant_contracts(transition_matrix=transition_matrix)
transition_matrix

In [ ]:
pfc_shifter._get_contract_start_end_dates()

In [ ]:
transition_matrix = pfc_shifter.generate_synthetic_contracts(transition_matrix=transition_matrix)
transition_matrix

In [ ]:
shifted = pfc_shifter.shift(transition_matrix)
shifted

In [ ]:
# relative error
rel_err = (df.loc[shifted.index, 'Spot'].values - shifted.iloc[:, 0].values)/df.loc[shifted.index, 'Spot'].values
plt.plot(rel_err)
plt.tight_layout()